Import Necessary Packages

In [1]:
# IF YOU HAVE A WINDOWS LAPTOP, PLEASE SPECIFIY WHERE YOU DOWNLOADED THE BIN FOLDER
# FOR OPENSLIDE

import os
OPENSLIDE_PATH = r'C:\Users\80027294\Documents\openslide-bin-4.0.0.3-windows-x64\openslide-bin-4.0.0.3-windows-x64\bin'

if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
        print("Imported openslide - from new path")
else:
    import openslide
import cv2
import numpy as np
from normalize_HnE import norm_HnE
import pdb
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import tifffile as tiff
#from utils import create_gt_tiles,retrieve_ann_vertex,retrieve_ann_coord
import imageio
# The path can also be read from a config file, etc.

Imported openslide - from new path


Change the following paths

In [2]:
# xml_path is the path to our xml annotations
xml_path = r'C:\Users\80027294\Documents\path-data-processing\annotations'
#wsi_path is the path to our whole slide image
wsi_path = r'C:\Users\80027294\Documents\path-data-processing\images'
# target_dir is the path to our target directory where we store our patches
# of our ground truth and our image
target_dir = r'C:\Users\80027294\Documents\path-data-processing\data_patches'

In [3]:
# DeepZoomGenerator method generates the patches we want based on specified size
from openslide.deepzoom import DeepZoomGenerator
from utils import create_gt_tiles,retrieve_ann_vertex,retrieve_ann_coord

Create patches, save patches, ground truth (mask) patches and patches that do not contain any annotations in seperate folders

In [4]:
i=0
# For every WSI in the directory of whole slide images,
for wsi in os.listdir(wsi_path):
    # CREATE PATCHES FROM ANNOTATIONS

    i+=1
    print('Step 1: Creation of Patch')
    print(f"Preprocessing Slide number {i} i.e {wsi}")
    slide = openslide.open_slide(os.path.join(wsi_path,wsi))

    # The path for the XML file that has the same name as the corresponding image file
    anno_path  = os.path.join(xml_path,wsi.replace(wsi[-3:],''))
    anno_path+='xml'
    
    # Parse the XML file that contains under the parent structure in root
    tree = ET.parse(anno_path)
    root = tree.getroot()

    # Define the dimensions for the mask (it should match the dimensions of the WSI)
    mask = np.zeros(slide.level_dimensions[0][::-1], dtype=np.uint8)
    if (wsi[-3:]) == 'svs':
        masked_image,mask = retrieve_ann_vertex(slide,root,mask)
    elif (wsi[-3:]=='tif'):
         masked_image,mask = retrieve_ann_coord(slide,root,mask)

    # BINARY MASK HAS BEEN CREATED, IF YOU WANT TO DISPLAY THE MASK
    # PLEASE UNCOMMENT ONE OF THE FOLLOWING COMMANDS

    # Display the masked image
    # plt.imshow(masked_image)
    # plt.imsave('masked_wsi.jpg',masked_image)
    # plt.imsave('mask.jpg',mask, cmap="gray")


    # CREATE PATCHES FROM OUR CREATED BINARY MASK AND WSI IMAGE
    print("Step 2: Converting to patches")
    # Generate object for tiles/patches using DeepZoomGenerator method of tile size 256
    tiles = DeepZoomGenerator(slide, tile_size=256, overlap=0, limit_bounds=False)
    tiles_gt = create_gt_tiles(mask, 256)
    #Here, we have divided our svs into tiles of size 256 with no overlap. 
    
    # We save our patches under the train_patches folder under our target_dir
    # We save our mask patches under the train_gt folder under our target_dir
    # We save the patches that do not contain annotations under the np folder under our target_dir
    wsi_name = target_dir+'/'+ 'train_patches/'
    wsi_name_gt = target_dir+'/'+ 'train_gt/'
    os.makedirs(wsi_name,exist_ok=True)
    os.makedirs(wsi_name_gt ,exist_ok=True)
    os.makedirs(target_dir+'/np',exist_ok=True)
    
    #pick the high resolution tiles
    # Indexing for processing our patches/tiles
    max_dim = len(tiles.level_tiles)
    cols, rows = tiles.level_tiles[max_dim-1]   
    tile_count = 0 

    for row in range(rows):
        for col in range(cols):
            tile_name = str(row) + "_" + str(col)
            temp_tile = tiles.get_tile(max_dim-1, (col, row))
            temp_tile_RGB = temp_tile.convert('RGB')
            temp_tile_np = np.array(temp_tile_RGB)
            #Save original tile

            if (temp_tile_np.mean() < 235 and temp_tile_np.std() > 15):

                print("Processing tile number:", tile_name)
                #tifffile giving weird bugs so i use imageio
                
                imageio.imwrite(wsi_name + wsi.replace(wsi[-4:],'') + tile_name + ".tif", temp_tile_np)
            
                plt.imsave(wsi_name_gt + wsi.replace(wsi[-4:],'') + tile_name + ".jpg",tiles_gt[tile_count][2],cmap="gray")
            else:
                print("NOT PROCESSING TILE:", tile_name)
                
                # Place the tiles with no annotations in the np folder 
                tiff.imwrite(target_dir+'/np' + '/' + tile_name + ".tif", temp_tile_np)
            
            tile_count+=1 # For iterating over our ground truth patches
    print(f"***Processed Slide{i}***")


Step 1: Creation of Patch
Preprocessing Slide number 1 i.e TMA457_1_1_C_.tif
Step 1/2: Retrieving Annotations
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
Multipolygon found! skipping..
WSI Dimensions: (2800, 2800)
Step 2: Converting to patches
NOT PROCESSING TILE: 0_0
